# Technical Portfolio Analyzer

This Jupyter Notebook will outline the exploration, developement and clean-up of the appliation code

### Table of Contents
- Data connectors
- Data query and clean-up
- Analytical methods
    - Technical
    - Fundamental
    - Traditional
    
---

- Visualization
- Dashboard (GUI)
    - GUI
    - Data filtration methods

### Dependencies

In [25]:
# Data analytics
import pandas as pd
import numpy as np
import panel as pn
# import bs4 as bs

# Visualization
# pn.extension('plotly')
# import plotly.express as px
# import hvplot.pandas
import matplotlib.pyplot as plt

# System
import os
import time
from pathlib import Path
from dotenv import load_dotenv
import requests

# Finance
import alpaca_trade_api as tradeapi
import quandl as ql

import warnings
warnings.filterwarnings('ignore')

## Data Connections
- Static Data Connections
- Dynamic Data Connections

#### Static Data Connections

In [2]:
# Get tickers within S&P500 index - test dataset
sp500_tickers_path = Path('resources/sp500_tickers.csv')
sp500_tickers = pd.read_csv(sp500_tickers_path)
sp500_tickers.head()

,Symbol,Security,GICS�Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888)
3,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989


### Bond Data

In [32]:
# Treasury bonds
treasure_yield = ql.get("USTREASURY/YIELD")

# Test data
# treasure_yield.head()

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
1990-01-02,NaN,NaN,7.83,7.89,7.81,7.87,7.90,7.87,7.98,7.94,NaN,8.00
1990-01-03,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04
1990-01-04,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
1990-01-05,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
1990-01-08,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09


#### Dynamic Data Connections

In [3]:
# Alpaca API connector
load_dotenv("../resources/alpaca_keys.env")

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version = "v2"
)

type(alpaca_api_key)

str

In [4]:
# Crypto connector URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

# Build out the crypto connector here

## Data Parsing

### Stock Data

In [21]:
# Get prices for tickers withing a given index or sector
def stock_prices(tickers_df, start_date, end_date):
    '''Returns pd.DataFrame with prices for the given tickers
    
    ...
    
    Parameters
    ----------
    tickers : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
    
    
    Returns
    -------
    result_df : pd.DataFrame with securities price data
    '''
    
    # Get list of tickers from the tickers_df DataFrame
    tickers = list(tickers_df["Symbol"])
    
    # Parse start and end dates
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    
    # Connect to Alpaca API and get data
    """Condition handling: 
        a. Alpaca API 422 Client Error if more than 100 tickers are passed - COMPLETE
        b. Alpaca API data max row limit of 1000 - PENDING"""
    
    
    # a. Alpaca API condition handling, sending 100 tickers at a time
    # Declate a pd.DataFrame
    result_df = pd.DataFrame()
    tickers_n = 50
    
    for i in range(0, len(tickers), tickers_n):
        # Slice the ticker list into lists of 50 tickers
        sliced_tickers = tickers[i:i + tickers_n] 
        
        temp_df = api.get_barset(
        sliced_tickers,
        timeframe = "1D",
        start = start_date,
        end = end_date,
        limit = 1000).df

        # Append temporary dataframe to result_df
        result_df = pd.concat([result_df, temp_df], axis = "columns", join = "outer")
        time.sleep(0.1)
        
    return result_df
    
    
# Method test
# start_date = '2020-01-01'
# end_date = '2020-02-28'
# tickers_df = sp500_tickers
# stocks = stock_prices(tickers_df, start_date, end_date)
# stocks.to_csv("resources/price_data.csv")
# stocks.head()

A                                  AAL  \
                            open   high    low  close   volume   open   
time                                                                    
2020-01-02 05:00:00+00:00  85.90  86.35  85.20  85.95  1199810  28.98   
2020-01-03 05:00:00+00:00  84.67  85.33  84.50  84.53   895182  28.27   
2020-01-06 05:00:00+00:00  84.00  84.82  83.60  84.78  1380173  27.19   
2020-01-07 05:00:00+00:00  83.96  85.26  83.94  85.09  1182275  27.56   
2020-01-08 05:00:00+00:00  85.96  86.47  85.20  85.91  1468876  27.10   

                                                            ...   ZION  \
                              high    low  close    volume  ...   open   
time                                                        ...          
2020-01-02 05:00:00+00:00  29.2950  28.65  29.09   5292996  ...  52.36   
2020-01-03 05:00:00+00:00  28.2900  27.34  27.65  13006571  ...  51.41   
2020-01-06 05:00:00+00:00  27.4901  27.08  27.32   5383583  ...  51.08   
2020-01-07 05:00:00+00:00  27.6800  27.06  27.22   5472322  ...  50.79   
2020-01-08 05:00:00+00:00  28.0900  27.07  27.84   9704511  ...  50.90   

                                                              ZTS           \
                             high     low  close   volume    open     high   
time                                                                         
2020-01-02 05:00:00+00:00  52.480  51.790  52.44  1307412  132.05  134.280   
2020-01-03 05:00:00+00:00  51.890  51.160  51.70  1012303  132.48  134.910   
2020-01-06 05:00:00+00:00  51.680  50.805  51.07  1048073  133.78  134.065   
2020-01-07 05:00:00+00:00  51.125  50.610  50.74  1214818  133.00  134.810   
2020-01-08 05:00:00+00:00  51.490  50.630  51.26  2102792  133.76  135.270   

                                                     
                              low    close   volume  
time                                                 
2020-01-02 05:00:00+00:00  131.48  134.150  1308668  
2020-01-03 05:00:00+00:00  132.27  134.110  1038786  
2020-01-06 05:00:00+00:00  132.71  133.100  1259478  
2020-01-07 05:00:00+00:00  132.67  133.600  1042889  
2020-01-08 05:00:00+00:00  133.24  133.325  1385439  

[5 rows x 2525 columns]

## Computational Methods

### Technical

In [ ]:
# please start with RSI and MACD
# your method must take entire dataframe and add a RSI or MACD column to it
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)


def rsi(df, days):
    '''Returns a pd.DataFrame with RSI column appended
    
    ...
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    days : int() - numbers of days for RSI calcualtion
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with RSI column appended, calcualted daily for 
        timeperiod specified by days
    '''
    
    result_df = pd.DataFrame()
    
    # Your code
    
    return result_df

def macd(df, short_ema, long_ema):
    '''Description...
    
    ...
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    short_ema : int() - short-term EMA for MACD calculation
    long_ema : int() - long-term EMA for MACD calculation
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with MACD column appended, calcualted daily for 
        timeperiod specified by days
    '''
    pass

# Method test

### Fundamental

In [ ]:
# Please start with PE, EPS, and Dividned Info.
# your method must take entire dataframe and add a PE, EPS, and Dividned Info columns to it
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)



# Method test

### Traditional

In [ ]:
# Please start with calculating the Sharpe Ratio Calculation
# your method must take entire dataframe and return a dataframe with ticker and sharpe ratio for the given time
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)

# Method test

## Visualization

In [ ]:
# your code here

## Dashboard

In [ ]:
# your code here